# Biosynthesis of Psylocybin in <i>Aspergillus niger</i>

## 1. Introduction

### 1.1 Literature review of the compound (<500 words)

### 1.2 Literature review of the cell factory (<500 words)

## 2. Problem definition (<300 words)

## 3. *If Project category I:* Reconstruction of a new GSM for your cell factory host of interest (<1500 words)

or

## 3. *If Project category II:* Selection and assessment of existing GSM (<500 words)

## 4. Computer-Aided Cell Factory Engineering (<1500 words if Category II project; <500 words for Category I project)

In [68]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

model = read_sbml_model('2018_ATCC1015_GSM.xml') #the model was found in (Brandl, J., Aguilar-Pontes, M.V., Schäpe, P. et al. - additional file 2) 

'' is not a valid SBML 'SId'.


In [69]:
model.medium

{'BOUNDARY_GLCe': 10.0,
 'BOUNDARY_H2Oe': 1000.0,
 'BOUNDARY_HNO3e': 1000.0,
 'BOUNDARY_O2e': 1000.0,
 'BOUNDARY_PIe': 1000.0,
 'BOUNDARY_SLFe': 1000.0,
 'BOUNDARY_Ke': 1000.0,
 'BOUNDARY_CAe': 1000.0}

#### Making the reactions

In [70]:
from cobra import Reaction, Metabolite

In [71]:
model.metabolites.TRP # tjekker lige at tryptophan befinder sig i cytosol

Metabolite identifier,TRP
Name,L-Tryptophan
Memory address,0x07f0437e6e280
Formula,C11H12N2O2
Compartment,c
In 8 reaction(s),"r385, r3008, r1080, r1266, r2402, PROTEINpolym, r400, r1265"


Jeg kan ikke finde tryptamin i modellen allerede (hverken som THR, THF, TRE, THY, TYR eller TPHTH og jeg synes ikke der er andre oplagte end dem jeg kan se - derfor tilføjer jeg sekv tryptamin som en metabolit:

Så derfor insættes metabolitterne også i compartment c (nedenfor)

Spørgsmål: burde vi også tilføje formula + name til metabolitterne?


In [72]:
tryptamine = Metabolite(id = 'TRYP', compartment='c')

In [73]:
hydroxytryptamine = Metabolite(id='_4HTRYP', compartment='c') 

In [74]:
norbaeocystin = Metabolite(id='NBC' , compartment='c') 

In [75]:
psilocybin = Metabolite(id='psi', compartment='c')

In [76]:
S_adenosyl_L_methionine = Metabolite(id='SAM', compartment='c')

In [77]:
S_adenosyl_L_homocysteine = Metabolite(id='SAH', compartment='c')

Først definerer jeg lige reaktionerne:

In [78]:
reaction_1 = Reaction('something_nice')  

In [79]:
reaction_2 = Reaction('something_nicer')

In [80]:
reaction_3 = Reaction('something_awesome')

In [81]:
reaction_4 = Reaction('Bjarne')

Nu tilføjer jeg alle metabolitterne til reaktionerne:

In [82]:
reaction_1.add_metabolites(({model.metabolites.TRP: -1,  
                              tryptamine: 1, 
                              model.metabolites.CO2: 1, 
                            }))

In [83]:
reaction_2.add_metabolites({tryptamine: -1,
                              model.metabolites.O2: -1,
                              model.metabolites.H: -2, 
                              hydroxytryptamine: 1, 
                              model.metabolites.H2O: 1
                             })

In [84]:
reaction_3.add_metabolites({hydroxytryptamine: -1,
                              model.metabolites.ATP: -1,
                              norbaeocystin: 1, 
                              model.metabolites.ADP: 1, # our newly created metabolites
                              model.metabolites.H: 1
                             })

In [85]:
reaction_4.add_metabolites(({norbaeocystin: -1,
                             S_adenosyl_L_methionine: -2, 
                             psilocybin: 1, 
                             S_adenosyl_L_homocysteine: 2,
                             model.metabolites.H: 2
                             }))

In [86]:
model.add_reactions([reaction_1])
model.add_reactions([reaction_2])
model.add_reactions([reaction_3])
model.add_reactions([reaction_4])

In [87]:
model.add_boundary(model.metabolites.TRYP, type='demand')
model.add_boundary(model.metabolites._4HTRYP, type='demand')
model.add_boundary(model.metabolites.NBC, type='demand')
model.add_boundary(model.metabolites.psi, type='demand')
model.add_boundary(model.metabolites.SAM, type='demand')
model.add_boundary(model.metabolites.SAH, type='demand')

Reaction identifier,DM_SAH
Name,S-Adenosyl-L-homocysteine demand
Memory address,0x07f0435837070
Stoichiometry,SAH --> S-Adenosyl-L-homocysteine -->
GPR,
Lower bound,0
Upper bound,1000.0


In [88]:
with model:
    model.objective = model.reactions.something_nice
    max_1 = model.optimize().objective_value
    
max_1

3.870967741935476

In [89]:
with model:
    model.objective = model.reactions.something_nicer
    max_2 = model.optimize().objective_value
    
max_2

3.5294117647058663

In [90]:
with model:
    model.objective = model.reactions.something_awesome
    max_3 = model.optimize().objective_value
    
max_3

3.529411764705867

In [91]:
with model:
    model.objective = model.reactions.Bjarne
    max_4 = model.optimize().objective_value
    
max_4

3.1395808620008148

In [92]:
for gene in model.genes.query('trp', 'name'):
    print(gene.name)

trpC
trpB homolog


In [93]:
essentiality = {}
for gene in model.genes:
    with model:
        gene.knock_out()
        essentiality[gene] = model.slim_optimize(error_value=0.)

In [94]:
essentiality2 = {}
for gene in model.genes:
    with model:
        gene.knock_out()
        essentiality2[gene] = model.optimize()

In [95]:
import operator

In [96]:
max(essentiality, key=essentiality.get)

Gene identifier,1144613
Name,adk1
Memory address,0x07f0437d2f7f0
Functional,True
In 7 reaction(s),"r543, r542, r436, r486, r491, r444, r541"


In [97]:
def keywithmaxval(d):
     """ a) create a list of the dict's keys and values; 
         b) return the key with the max value"""  
     v=list(d.values())
     k=list(d.keys())
     return k[v.index(max(v))]

In [98]:
keywithmaxval(essentiality)

Gene identifier,1144613
Name,adk1
Memory address,0x07f0437d2f7f0
Functional,True
In 7 reaction(s),"r543, r542, r436, r486, r491, r444, r541"


In [99]:
all_values = essentiality.values()
max_value = max(all_values)



print(max_value)

0.9398547241513246


In [100]:
with model as pgk_mutant:
    pgk_mutant.genes.b2926.knock_out()
    print(pgk_mutant.slim_optimize())

AttributeError: DictList has no attribute or entry b2926

In [ ]:
with model:
    model.objective = model.reactions.r385
    print(model.optimize().objective_value)

In [ ]:
with model:
        model.objective = model.reactions.r385 
        for gene in model.genes:
            gene.knock_out()
            if model.optimize().objective_value > 48.2:
                print(gene)

In [ ]:
for gene in model.genes:
    with model:
        gene.knock_out()
        model.objective = model.reactions.r385 
        if model.optimize().objective_value > 48:
            print(gene)

In [ ]:
for reaction in model.metabolites.TRP.reactions:
    print(reaction.id, reaction, reaction.name)

In [ ]:
for reaction in model.reactions.query('total', 'name'):
    print(reaction.name)

In [ ]:
model.reactions.DRAIN_Biomass

In [ ]:
model.reactions.r385.flux


In [101]:
knockout_fluxes = {}
for gene in model.genes:
    with model:
        gene.knock_out()
        solution = model.reactions.r385
        if model.slim_optimize(error_value=0.) != 0:
            knockout_fluxes[gene] = solution.flux

knockout_fluxes

{<Gene 1128503 at 0x7f0437d72c70>: 0.0311449058488579,
 <Gene 1145456 at 0x7f0437d72dc0>: 0.0311449058488579,
 <Gene 1155770 at 0x7f0437d720a0>: 0.0311449058488579,
 <Gene 1143538 at 0x7f0437d72d00>: 0.0311449058488579,
 <Gene 1146031 at 0x7f0437d72cd0>: 0.03067545946538437,
 <Gene 1126547 at 0x7f0437d72040>: 0.031144905848858106,
 <Gene 1128684 at 0x7f0437d72070>: 0.03014894565548194,
 <Gene 1146884 at 0x7f0437d72df0>: 0.031144905848858002,
 <Gene 1145051 at 0x7f0437d72e20>: 0.03014894565548192,
 <Gene 1141976 at 0x7f0437d72e50>: 0.031144905848858005,
 <Gene 1144773 at 0x7f0437d72e80>: 0.031144905848858005,
 <Gene 1145793 at 0x7f0437d72eb0>: 0.02773285600493612,
 <Gene 1148178 at 0x7f0437d72ee0>: 0.02773285600493612,
 <Gene 1145895 at 0x7f0437d72f10>: 0.031144905848857672,
 <Gene 1124768 at 0x7f0437d72f40>: 0.031144905848857672,
 <Gene 1183468 at 0x7f0437d72f70>: 0.031144905848857672,
 <Gene 1113563 at 0x7f0437d72fa0>: 0.028397000115324533,
 <Gene 1184650 at 0x7f0437d72fd0>: 0.0284191

In [107]:
knockout_fluxes1 = {}
for gene in model.genes:
    with model:
        gene.knock_out()
        solution = model.reactions.r385
        if solution.flux != 0:
            knockout_fluxes1[gene] = solution.flux

knockout_fluxes1

{<Gene 1128503 at 0x7f0437d72c70>: 0.0311449058488578,
 <Gene 1145456 at 0x7f0437d72dc0>: 0.0311449058488578,
 <Gene 1155770 at 0x7f0437d720a0>: 0.0311449058488578,
 <Gene 1143538 at 0x7f0437d72d00>: 0.0311449058488578,
 <Gene 1146031 at 0x7f0437d72cd0>: 0.0311449058488578,
 <Gene 1126547 at 0x7f0437d72040>: 0.0311449058488578,
 <Gene 1128684 at 0x7f0437d72070>: 0.0311449058488578,
 <Gene 1146884 at 0x7f0437d72df0>: 0.0311449058488578,
 <Gene 1145051 at 0x7f0437d72e20>: 0.0311449058488578,
 <Gene 1141976 at 0x7f0437d72e50>: 0.0311449058488578,
 <Gene 1144773 at 0x7f0437d72e80>: 0.0311449058488578,
 <Gene 1145793 at 0x7f0437d72eb0>: 0.0311449058488578,
 <Gene 1148178 at 0x7f0437d72ee0>: 0.0311449058488578,
 <Gene 1145895 at 0x7f0437d72f10>: 0.0311449058488578,
 <Gene 1124768 at 0x7f0437d72f40>: 0.0311449058488578,
 <Gene 1183468 at 0x7f0437d72f70>: 0.0311449058488578,
 <Gene 1113563 at 0x7f0437d72fa0>: 0.0311449058488578,
 <Gene 1184650 at 0x7f0437d72fd0>: 0.0311449058488578,
 <Gene 103

In [108]:
keywithmaxval(knockout_fluxes1)

Gene identifier,1128503
Name,56084
Memory address,0x07f0437d72c70
Functional,True
In 4 reaction(s),"r2, r1, r4, r3"


In [111]:
for gene in knockout_fluxes1:
    if gene == 1128503:
        print(gene)

In [103]:
model.reactions.r376.flux

0.1277929866972975

## 5. Discussion (<500 words)

## 6. Conclusion (<200 words)

## References
Brandl, J., Aguilar-Pontes, M.V., Schäpe, P. et al. A community-driven reconstruction of the Aspergillus niger metabolic network. Fungal Biol Biotechnol 5, 16 (2018). https://doi.org/10.1186/s40694-018-0060-7